# Street names of Madrid

## Scraping list names from the council web

In [1]:
from requests import get

### Scraping street names that begin with A

In [2]:
url = "http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=A"
response = get(url)
print(response.text[:500])


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="es">

<head>
	<meta http-equiv="Content-Type" content="text/html; charset=windows-1252"/>
	<meta name="TITLE" content="Nomecalles - Nomenclator y Callejero de la Comunidad de Madrid"/>
	<meta name="DC.TITLE" content="Nomecalles - Nomenclator y Callejero de la Comunidad de Madrid"/>
	<meta http-equiv="TITLE" content="No


In [3]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
street_container = html_soup.find_all('td', attrs = {'width': '25%'})

In [5]:
street_names = street_container[0].find_all('a')

In [6]:
len(street_names)

250

In [16]:
a_streets = []

for n in range(len(street_names)):
    name = street_names[n].text
    a_streets.append(name)

a_streets[:10]

['Calle de Abad Juan Catalán',
 'Calle de Abada',
 'Calle de Abades',
 'Calle de la Abadesa',
 'Calle de Ábalos',
 'Calle del Abanico',
 'Calle de Abantos',
 'Calle de Abardero',
 'Calle de Abarzuza',
 'Calle de Abay']

### Scraping all the letters

In [12]:
container = html_soup.find_all('td')
letter_container = container[5].find_all('a')

In [15]:
urls = []

for a in letter_container:
    url = a.get('href')
    urls.append('http://www.madrid.org'+url)
    
urls[:10]

['http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=A',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=B',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=C',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=D',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=E',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=F',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=G',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=H',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=I',
 'http://www.madrid.org/nomecalles/ListaCalles.icm?munic=079&letra_inicial=J']

In [17]:
streets_final = []

for url in urls:
    response = get(url)
    if response.status_code != 200:
        continue
    html_soup = BeautifulSoup(response.text, 'html.parser')
    street_container = html_soup.find_all('td', attrs = {'width': '25%'})
    street_names = street_container[0].find_all('a')
    for n in range(len(street_names)):
        name = street_names[n].text
        streets_final.append(name)

streets_final[:5]

['Calle de Abad Juan Catalán',
 'Calle de Abada',
 'Calle de Abades',
 'Calle de la Abadesa',
 'Calle de Ábalos']

In [18]:
len(streets_final)

2275

In [19]:
import re

In [52]:
names = []

pattern = '\s([A-ZÀ-Ö].+)'

for st in streets_final:
        name = re.findall(pattern, st)
        if len(name) > 0:
            names.append(name[0])

names[:10]

['Abad Juan Catalán',
 'Abada',
 'Abades',
 'Abadesa',
 'Ábalos',
 'Abanico',
 'Abantos',
 'Abardero',
 'Abarzuza',
 'Abay']

In [53]:
len(names)

2272

In [54]:
import pandas as pd

streets_series = pd.Series(streets_final)

pattern = '\s([A-ZÀ-Ö].+)' #captura desde la primera letra mayúscula precedida por un espacio

names_series = streets_series.str.extract(pattern)

names_series.head()

,0
0,Abad Juan Catalán
1,Abada
2,Abades
3,Abadesa
4,Ábalos


In [55]:
len(names_series.notnull())

2275